In [1]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

**setup kaggle credentials**

In [2]:
data=pd.read_csv("../data/IMDB Dataset.csv")

In [3]:
data.shape

(50000, 2)

In [4]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
data.tail(10)

,review,sentiment
49990,"Lame, lame, lame!!! A 90-minute cringe-fest th...",negative
49991,"Les Visiteurs, the first movie about the medie...",negative
49992,John Garfield plays a Marine who is blinded by...,positive
49993,Robert Colomb has two full-time jobs. He's kno...,negative
49994,This is your typical junk comedy.<br /><br />T...,negative
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [6]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [7]:
sentiment_map = {"positive": 1, "negative": 0}

data["new_c"] = data["sentiment"].map(sentiment_map).fillna(-1) #add .fillna to handle values not in the map.


In [8]:
# data.replace({"sentiment" : {"positive":1 , "negative":0}}, inplace=True)

In [9]:
data.head()

,review,sentiment,new_c
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [10]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [11]:
data.head(10)

,review,sentiment,new_c
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1
5,"Probably my all-time favorite movie, a story o...",positive,1
6,I sure would like to see a resurrection of a u...,positive,1
7,"This show was an amazing, fresh & innovative i...",negative,0
8,Encouraged by the positive comments about this...,negative,0
9,If you like original gut wrenching laughter yo...,positive,1


In [12]:
#split
train_data, test_data =train_test_split(data, test_size=0.2, random_state=42)

In [13]:
train_data.shape

(40000, 3)

In [14]:
test_data.shape

(10000, 3)

**Tokenize Text data**

In [15]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [16]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [17]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [18]:
Y_test=test_data["sentiment"]

In [19]:
Y_train=train_data["sentiment"]

In [20]:
print(Y_test)
print(Y_train)

33553    positive
9427     positive
199      negative
12447    positive
39489    negative
           ...   
28567    negative
25079    positive
18707    positive
15200    negative
5857     positive
Name: sentiment, Length: 10000, dtype: object
39087    negative
30893    negative
45278    positive
16398    negative
13653    negative
           ...   
11284    positive
44732    positive
38158    negative
860      positive
15795    positive
Name: sentiment, Length: 40000, dtype: object


In [26]:
Y_train = Y_train.replace({"positive": 1, "negative": 0})
Y_test = Y_test.replace({"positive": 1, "negative": 0})

C:\Users\Aditya Raj\AppData\Local\Temp\ipykernel_2436\753937383.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_train = Y_train.replace({"positive": 1, "negative": 0})
C:\Users\Aditya Raj\AppData\Local\Temp\ipykernel_2436\753937383.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_test = Y_test.replace({"positive": 1, "negative": 0})


**Now, build the model**

In [27]:
model = Sequential()
model.add(Input(shape=(200,))) # Explicit input layer
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [28]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

**Compile the model**

In [29]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Train the model**

In [30]:
model.fit(X_train, Y_train, epochs=6, batch_size=64, validation_split=0.2)

Epoch 1/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 74s 145ms/step - accuracy: 0.7215 - loss: 0.5358 - val_accuracy: 0.8244 - val_loss: 0.4026
Epoch 2/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 66s 131ms/step - accuracy: 0.8499 - loss: 0.3599 - val_accuracy: 0.7441 - val_loss: 0.5061
Epoch 3/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 65s 131ms/step - accuracy: 0.8573 - loss: 0.3393 - val_accuracy: 0.8727 - val_loss: 0.3165
Epoch 4/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 66s 132ms/step - accuracy: 0.8919 - loss: 0.2696 - val_accuracy: 0.8725 - val_loss: 0.3128
Epoch 5/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 66s 132ms/step - accuracy: 0.9100 - loss: 0.2348 - val_accuracy: 0.8751 - val_loss: 0.3116
Epoch 6/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 70s 140ms/step - accuracy: 0.9086 - loss: 0.2340 - val_accuracy: 0.8802 - val_loss: 0.3170


**Model evaluation**

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8896 - loss: 0.2770
Test Loss: 0.2760275602340698
Test Accuracy: 0.8898000121116638


**Building a Predictive System**

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

**Let's take some example to check function**

In [ ]:
review1="movie is totally bad"
sentiment = predict_sentiment(review1)
print("sentiment=",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
sentiment= negative


In [ ]:
review2="only some part of movie is not good"
sentiment = predict_sentiment(review2)
print("sentiment=",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
sentiment= negative


In [ ]:
review3="only some part of movie is not bad"
sentiment = predict_sentiment(review3)
print("sentiment=",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
sentiment= negative


In [ ]:
review4="only some part of movie is perfect"
sentiment = predict_sentiment(review4)
print("sentiment=",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
sentiment= positive


In [ ]:
review5="whole movie was ok"
sentiment = predict_sentiment(review5)
print("sentiment=",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
sentiment= negative


In [ ]:
review5="everything is perfect"
sentiment = predict_sentiment(review5)
print("sentiment=",sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
sentiment= positive
